In [137]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dot, Softmax, Concatenate, Flatten
from tensorflow.keras.models import Model

In [10]:
import numpy as np

In [142]:
df = pd.read_pickle('../../data/datasets/raw-video-level-watches')
videos = pd.read_pickle('../../data/videos_raw_metadata')
title_embeddings = pd.read_pickle('../../data/embeddings/title-autoencoded')
tag_embeddings = pd.read_pickle('../../data/embeddings/tag_embeddings.pkl')
topic_embeddings = pd.read_pickle('../../data/embeddings/topic_embeddings.pkl')

In [143]:
class SiameseWithAttention(tf.keras.Model):
    def __init__(self, embedding_size, num_suggested_videos):
        super(SiameseWithAttention, self).__init__()
        
        self.shared_fc = tf.keras.Sequential([
            Dense(1024, activation='relu'),
            # Dense(512, activation='relu'),
            # Dense(256, activation='relu')
        ])       

        self.hidden_fc = Dense(256, activation='relu')
        self.flatten = Flatten()
        self.attention_weights = Dense(num_suggested_videos, activation='softmax')
        self.predictor = Dense(num_suggested_videos, activation='softmax')
        
    def call(self, inputs):
        playing_video = inputs[0]
        suggested_videos = inputs[1]
        
        playing_video_emb = self.shared_fc(playing_video)
        suggested_videos_emb = self.shared_fc(suggested_videos)
        
        attention_scores = self.attention_weights(playing_video_emb)
        weighted_suggested_videos = Dot(axes=1)([attention_scores, suggested_videos_emb])
        
        combined_emb = Concatenate()([playing_video_emb, weighted_suggested_videos])
        hidden = self.hidden_fc(self.flatten(combined_emb))
        predicted_scores = self.predictor(hidden)
        
        return predicted_scores

In [144]:
def trim(vec, element, length=15):
    #  a vector to length and keep the element
    if len(vec) > length:
        vec = vec[:length]
    if element not in vec:
        vec = vec[:-1] + [element]

    return vec

In [110]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2)

In [131]:
def get_data(df):
    playings = []
    SG = []
    selecteds = []

    for watch in df_train:
        y = []
        playing = watch['playing']
        selected = watch['selected']
        suggesteds = watch['upnext']

        suggesteds = trim(suggesteds, selected)
        
        try:
            if playing not in videos or selected not in videos or any([s not in videos for s in suggesteds]):
                continue

            p = np.array(title_embeddings[videos[playing]['snippet']['title']])
            sx = []
            for s in suggesteds:
                title = np.array(title_embeddings[videos[s]['snippet']['title']])
                sx.append(title)

            if len(sx) < 15:
                continue

            for s in suggesteds:
                if s == selected:
                    y.append(1)
                else:
                    y.append(0)

            playings.append(p)
            SG.append(sx)
            selecteds.append(y)
            
        except:
            continue

    # return playings, SG, selecteds
    return np.array(playings), np.array(SG), np.array(selecteds)

In [132]:
p, s, y = get_data(df_train)

In [ ]:
pp = tf.convert_to_tensor(p)
ss = tf.convert_to_tensor(s)
yy = tf.convert_to_tensor(y)

In [140]:
pp.shape

TensorShape([4059, 1536])

In [141]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dot, Softmax, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

embedding_size = 1536
num_suggested_videos = 15
model = SiameseWithAttention(embedding_size, num_suggested_videos)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
batch_size = 64
epochs = 100

model.fit([pp, ss],
          yy,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1)


Epoch 1/100
58/58 [==============================] - 2s 10ms/step - loss: 2.3524 - accuracy: 0.3208 - val_loss: 2.3657 - val_accuracy: 0.3079
Epoch 2/100
58/58 [==============================] - 0s 7ms/step - loss: 2.3129 - accuracy: 0.3260 - val_loss: 2.4468 - val_accuracy: 0.3079
Epoch 3/100
58/58 [==============================] - 0s 7ms/step - loss: 2.2941 - accuracy: 0.3260 - val_loss: 2.3771 - val_accuracy: 0.3079
Epoch 4/100
58/58 [==============================] - 0s 7ms/step - loss: 2.2884 - accuracy: 0.3260 - val_loss: 2.4924 - val_accuracy: 0.3079
Epoch 5/100
58/58 [==============================] - 0s 7ms/step - loss: 2.2603 - accuracy: 0.3258 - val_loss: 2.3920 - val_accuracy: 0.3079
Epoch 6/100
58/58 [==============================] - 0s 7ms/step - loss: 2.2245 - accuracy: 0.3247 - val_loss: 2.4056 - val_accuracy: 0.2882
Epoch 7/100
58/58 [==============================] - 0s 7ms/step - loss: 2.1743 - accuracy: 0.3408 - val_loss: 2.4340 - val_accuracy: 0.2660
Epoch 8/100


KeyboardInterrupt: 

In [76]:
p.shape

(4043, 128)

In [77]:
s.shape

(4043, 15, 128)